In [1]:
# set the matplotlib backend so figures can be saved in the background
#import matplotlib
#matplotlib.use("Agg")
#import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers import Dense, Input
from sklearn.metrics import classification_report
from keras.layers import Dense, LSTM

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
#from google.colab.patches import cv2_imshow
#from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2 as cv 
import os
import json
import pickle
import tensorflow as tf
from tensorflow import keras

In [2]:
### JOINTS FEATCHERS CODE #####


path='E:/two stream final/'
import numpy as np
file1=open(path+"jointsFile","rb")
file2 = open(path+"levelFile", "rb")
with open(path+'featcherFile', 'rb') as file3:
    featchers_array=pickle.load(file3)
    
joints_array= np.load(file1)
labels= np.load(file2)

print("joints_array=",joints_array.shape)
print("labels=",labels.shape)
print("featchers_array",featchers_array.shape)


joints_array= (11082, 72, 25, 3)
labels= (11082, 100)
featchers_array (11082, 7, 72)


In [3]:
(trainFeatcherX, testFeatcherX, trainFeLabelY, testFeLabelY) = train_test_split(featchers_array,labels,
	test_size=0.25 , random_state=42)
print(trainFeatcherX.shape)

(8311, 7, 72)


In [4]:
# define our LSTM network
model1 = keras.Sequential()
model1.add(keras.Input(shape=(7,72)))
model1.add(LSTM(32))
model1.add(Dense(8, activation="relu"))
model1.add(Dense(4, activation="relu"))
model1.add(Dense(1, activation="linear"))


   


In [7]:
#### OPTICAL FLOW CODE#####
with open(path+'Label.data', 'rb') as l:
    Optical_image_lables=pickle.load(l)
    
with open(path+'ALL_OPTICAL_IMAGE.data', 'rb') as f:
    optical_images=pickle.load( f)


### ONE HOT ENCODING ####
Optical_image_lables = np.asarray(Optical_image_lables)
#lb = LabelBinarizer()
#Optical_image_lables = lb.fit_transform(Optical_image_lables)

print("optical_images=",optical_images.shape)
print("Optical_image_lables=",Optical_image_lables.shape)


optical_images= (13197, 240, 320, 3)
Optical_image_lables= (13197,)


In [13]:
(trainImageX, testImageX, trainImageLabelY, tesImageLabeltY) = train_test_split(optical_images, Optical_image_lables,
	test_size=0.25, stratify=Optical_image_lables, random_state=20)

print(trainImageX.shape)

MemoryError: Unable to allocate 8.49 GiB for an array with shape (9897, 240, 320, 3) and data type float32

In [3]:
from keras.utils import np_utils
trainImageLabelY = np_utils.to_categorical(trainImageLabelY)
tesImageLabeltY = np_utils.to_categorical(tesImageLabeltY)
print(trainImageLabelY.shape)

NameError: name 'trainImageLabelY' is not defined

In [ ]:
trainImageX=trainImageX[:8311]
trainImageLabelY=trainImageLabelY[:8311]
print(trainImageX.shape)

In [13]:
# define our CNN network
model2_in=Input(shape=(240, 320, 3))

baseModel = ResNet50(weights="imagenet", include_top=False,
     input_tensor=model2_in)
# construct the head of the model that will be placed on top of the
# the base model
model2_out = baseModel.output
model2_out = AveragePooling2D(pool_size=(7, 7))(model2_out)
model2_out = Flatten(name="flatten")(model2_out)
model2_out = Dense(512, activation="relu")(model2_out)
model2_out = Dropout(0.5)(model2_out)
model2_out=Dense(1, activation="linear")(model2_out)


model2 = Model(inputs=model2_in, outputs=model2_out)

  

In [14]:
#### COMBINED TWO MODELS ###
combined = concatenate([model1.output, model2.output])
z = Dense(2, activation="relu")(combined)
z = Dense(1, activation="linear")(z)
model = Model(inputs=[model1.input, model2.input], outputs=z)

In [ ]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
# train the model
print("[INFO] training model...")
model.fit(
    [trainFeatcherX, trainImageX],trainImageLabelY,
    #validation_data=([testAttrX, testImagesX], testY),
    epochs=10, batch_size=8)
# make predictions on the testing data
print("[INFO] predicting house prices...")
#preds = model.predict([testAttrX, testImagesX])

[INFO] training model...
Epoch 1/10
 755/1039 [====================>.........] - ETA: 25:41 - loss: 132041.9029